<a href="https://colab.research.google.com/github/phuri7987/229352-StatisticalLearning-Lab/blob/main/Lab03_kNN_using_Grid_Search_640510703.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

In [1]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


In [2]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

X[0]: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





y[0]: 7


In [3]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define transformation
tfidf = TfidfVectorizer(stop_words = 'english',
                        ngram_range = (2, 2))

# fit+transform training set
Xtrain_tfidf = tfidf.fit_transform(Xtrain)

# See output
Xtrain_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 461825 stored elements and shape (3000, 351473)>

In [5]:
#tfidf.vocabulary_

#### Exercise 1: Find post in the training set that is closest in tf-idf to the first post in the test set (`Xtest[0]`). Print the content of both posts (not the tf-idf vectors).

### Classify with k-Nearest Neighbor (kNN) ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html))

In [6]:
from sklearn.neighbors import KNeighborsClassifier

# Define model
nb = KNeighborsClassifier(n_neighbors=5)

# Fit the model
nb.fit(Xtrain_tfidf, ytrain)

KNeighborsClassifier()

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [7]:
from sklearn.metrics import classification_report

# Transform the test set
Xtest_tfidf = tfidf.transform(Xtest)

# Make predictions on the test set
ypred = nb.predict(Xtest_tfidf)

#report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.47      0.67      0.55        21
           1       0.24      0.38      0.29        21
           2       0.23      0.31      0.26        26
           3       0.31      0.29      0.30        34
           4       0.27      0.26      0.27        34
           5       0.48      0.42      0.45        26
           6       0.30      0.41      0.35        22
           7       0.48      0.39      0.43        28
           8       0.59      0.67      0.63        33
           9       0.59      0.64      0.62        25
          10       0.33      0.33      0.33        27
          11       0.76      0.80      0.78        20
          12       0.22      0.08      0.12        24
          13       0.47      0.39      0.43        23
          14       0.63      0.61      0.62        28
          15       0.83      0.52      0.64        29
          16       0.59      0.62      0.60        21
          17       0.67    

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [9]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words = 'english')),
                     ('nb', KNeighborsClassifier())])

# Fit the pipeline to the training set
pipeline.fit(Xtrain, ytrain)

# Make predictions on the test set
ypred = pipeline.predict(Xtest)

# report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.35      0.43      0.38        21
           1       0.33      0.48      0.39        21
           2       0.46      0.62      0.52        26
           3       0.45      0.56      0.50        34
           4       0.62      0.47      0.53        34
           5       0.88      0.54      0.67        26
           6       0.35      0.55      0.43        22
           7       0.53      0.57      0.55        28
           8       0.80      0.73      0.76        33
           9       0.77      0.68      0.72        25
          10       0.67      0.74      0.70        27
          11       0.76      0.95      0.84        20
          12       0.36      0.21      0.26        24
          13       0.78      0.61      0.68        23
          14       0.84      0.75      0.79        28
          15       0.62      0.62      0.62        29
          16       0.56      0.71      0.62        21
          17       0.82    

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('knn', KNeighborsClassifier())
])

params = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'knn__n_neighbors': [3, 5, 7],
    'knn__metric': ['cosine', 'euclidean']
}

from sklearn.model_selection import GridSearchCV

gridcv = GridSearchCV(pipeline, params, scoring='f1_macro', cv=5)
gridcv.fit(Xtrain, ytrain)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['cosine', 'euclidean'],
                         'knn__n_neighbors': [3, 5, 7],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_macro')

In [21]:
gridcv.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('knn', KNeighborsClassifier(metric='cosine', n_neighbors=7))])

In [22]:
# Make predictions on the test set
ypred = gridcv.predict(Xtest)

# Report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.45      0.48      0.47        21
           1       0.46      0.57      0.51        21
           2       0.48      0.58      0.53        26
           3       0.45      0.53      0.49        34
           4       0.67      0.65      0.66        34
           5       0.73      0.62      0.67        26
           6       0.33      0.45      0.38        22
           7       0.60      0.64      0.62        28
           8       0.68      0.70      0.69        33
           9       0.80      0.64      0.71        25
          10       0.66      0.78      0.71        27
          11       0.70      0.95      0.81        20
          12       0.44      0.17      0.24        24
          13       0.78      0.61      0.68        23
          14       0.78      0.75      0.76        28
          15       0.61      0.66      0.63        29
          16       0.56      0.67      0.61        21
          17       0.78    

#### Exercise 2:

1. Use grid search 5-fold cross-validation across different values of the following two kNN parameters: `n_neighbors` and `metric`  **on the training set** to find the best model.

2. For the best value of `n_neighbors` and `metric` you found above, compute the `f1_macro` score **on the test set**.
* Print the value of `n_neighbors` and `metric`.
* Print the model's `f1_macro` score.

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('knn', KNeighborsClassifier())
])

params = {
    'knn__n_neighbors': [3, 5, 7],
    'knn__metric': ['euclidean', 'manhattan', 'cosine']
}

gridcv = GridSearchCV(
    estimator=pipeline,
    param_grid=params,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1
)

gridcv.fit(Xtrain, ytrain)

print("Best parameters:", gridcv.best_params_)
print("Best CV score:", gridcv.best_score_)


Best parameters: {'knn__metric': 'euclidean', 'knn__n_neighbors': 7}
Best CV score: 0.685913288216576


In [25]:
from sklearn.metrics import f1_score

best_n_neighbors = gridcv.best_params_['knn__n_neighbors']
best_metric = gridcv.best_params_['knn__metric']

print(f"Best n_neighbors: {best_n_neighbors}")
print(f"Best metric: {best_metric}")


best_knn = KNeighborsClassifier(n_neighbors=best_n_neighbors, metric=best_metric)


from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,1))


Xtrain_tfidf = tfidf.fit_transform(Xtrain)
Xtest_tfidf = tfidf.transform(Xtest)


best_knn.fit(Xtrain_tfidf, ytrain)


y_pred = best_knn.predict(Xtest_tfidf)
f1_macro_score = f1_score(ytest, y_pred, average='macro')

print(f"F1 macro score on test set: {f1_macro_score:.4f}")

Best n_neighbors: 7
Best metric: euclidean
F1 macro score on test set: 0.5911
